In [1]:
import keras
import keras.backend as K

/Users/user/anaconda3/envs/ml_py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Copy from https://github.com/duggalrahul/AlexNet-Experiments-Keras/blob/master/convnets-keras/convnetskeras/convnets.py

In [65]:
def LRN_layer(alpha=1e-4, beta=0.75, n=5, k=2,**kwargs):
    from keras.layers.core import Lambda
    def LRN_function(X):
        import keras.backend as K
        batch, height, width, channel = X.shape
        square = K.square(X)
        extra_channels = make_2d_padding_works(K.permute_dimensions(square, (0,2,1,3)),(0, n//2))
        extra_channels = K.permute_dimensions(extra_channels, (0,1,3,2))
        
        scale = k
        
        for i in range(n):
            print(i)
            scale += alpha * extra_channels[:, i:i + channel, :, :]
        scale = scale ** beta
        return X / scale
    
    def make_2d_padding_works(x, padding, data_format=None):
        return K.spatial_2d_padding(x,((padding[0],padding[0]),(padding[1],padding[1])),data_format)
    
    return Lambda(LRN_function, output_shape=lambda input_shape:input_shape, **kwargs)

In [66]:
def splittensor(axis=1, ratio_split=1, id_split=0,**kwargs):
    def f(X):
        div = X.shape[axis] // ratio_split

        if axis == 0:
            output =  X[id_split*div:(id_split+1)*div,:,:,:]
        elif axis == 1:
            output =  X[:, id_split*div:(id_split+1)*div, :, :]
        elif axis == 2:
            output = X[:,:,id_split*div:(id_split+1)*div,:]
        elif axis == 3:
            output = X[:,:,:,id_split*div:(id_split+1)*div]
        else:
            raise ValueError("This axis is not possible")

        return output

    def g(input_shape):
        output_shape=list(input_shape)
        output_shape[axis] = output_shape[axis] // ratio_split
        return tuple(output_shape)

    return Lambda(f,output_shape=lambda input_shape:g(input_shape),**kwargs)

In [95]:
from keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import merge, Flatten, Dense, Dropout
from keras.models import Sequential, Model

In [96]:
def define_alexnet():
    inputs = Input(shape=(227,227,3))
    
    conv_1 = Conv2D(96, 11, 11, subsample=(4, 4), activation='relu',name='conv_1')(inputs)
    conv_1 = MaxPooling2D((3, 3), strides=(2, 2))(conv_1)
    conv_1 = LRN_layer(name="convpool_1")(conv_1)
    conv_1 = ZeroPadding2D((2,2))(conv_1)
    
    conv_2 = merge([
        Conv2D(128,kernel_size=(5,5), strides=5, activation='relu', name='conv2_'+str(i+1))(
            splittensor(ratio_split=2, id_split=1)(conv_2)
        ) for i in range(2)], mode='concat', concat_axis=1, name="conv_2"
    )

In [97]:
def define_fake_alexnet():
    inputs = Input(shape=(227,227,3))
    
    conv_1 = Conv2D(96, 11, 11, subsample=(4, 4), activation='relu',name='conv_1')(inputs)
    conv_1 = MaxPooling2D((3, 3), strides=(2, 2))(conv_1)
    conv_1 = ZeroPadding2D((2,2))(conv_1)
    
    conv_2 = Conv2D(128,5,5,activation='relu')(conv_1)
    conv_2 = MaxPooling2D((3,3), strides=(2, 2))(conv_2)
    conv_2 = ZeroPadding2D((1,1))(conv_2)
    
    conv_3 = Conv2D(384, 3, 3, activation='relu')(conv_2)
    conv_4 = ZeroPadding2D((1,1))(conv_3)
    
    conv_4 = Conv2D(192,3,3,activation='relu')(conv_3)
    conv_4 = ZeroPadding2D((1,1))(conv_4)
    
    conv_5 = Conv2D(128,3,3,activation='relu')(conv_4)
    conv_5 = MaxPooling2D((3, 3), strides=(2, 2))(conv_5)
    
    dense_1 = Flatten()(conv_5)
    dense_1 = Dense(4096, activation='relu')(dense_1)
    dense_1 = Dropout(0.5)(dense_1)
    
    dense_2 = Dense(4096, activation='relu')(dense_1)
    dense_2 = Dropout(0.5)(dense_2)
    
    dense_3 = Dense(1000,activation='softmax')(dense_2)
    
    return Model(input=inputs, output=dense_3)
    

In [98]:
model = define_fake_alexnet()

/Users/user/anaconda3/envs/ml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (11, 11), activation="relu", name="conv_1", strides=(4, 4))`
  after removing the cwd from sys.path.
/Users/user/anaconda3/envs/ml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu")`
  
/Users/user/anaconda3/envs/ml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(384, (3, 3), activation="relu")`
  if sys.path[0] == '':
/Users/user/anaconda3/envs/ml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (3, 3), activation="relu")`
  from ipykernel import kernelapp as app
/Users/user/anaconda3/envs/ml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Upda

In [99]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 227, 227, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_21 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 27, 27, 128)       307328    
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 13, 13, 128)       0         
_________________________________________________________________
zero_padding2d_22 (ZeroPaddi (None, 15, 15, 128)       0         
__________

In [69]:
define_alexnet()

(?, 27, 96, 31)
0
1


/Users/user/anaconda3/envs/ml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (11, 11), activation="relu", name="conv_1", strides=(4, 4))`
  after removing the cwd from sys.path.


ValueError: Dimensions must be equal, but are 27 and 26 for 'convpool_1_13/add_1' (op: 'Add') with input shapes: [?,27,96,31], [?,26,96,31].